In [112]:
import pandas as pd
import numpy as np
import random
import re

In [2]:
NUM_ENCOUNTERS = 1000;
MONSTER_NUM_COL = 1;
MONSTER_HDN_COL = 12;
MONSTER_EHD_COL = 13

In [8]:
mainTable = pd.read_csv('../../WildMainTable.csv')
mainTable

,Die,Clear,Woods,River,Swamp,Mountain,Desert,City
0,1,Men,Men,Men,Men,Men,Men,Men
1,2,Flyer,Flyer,Flyer,Flyer,Flyer,Flyer,Undead
2,3,Giant,Giant,Giant,Giant,Giant,Giant,Undead
3,4,Lycan,Lycan,Lycan,Lycan,Lycan,Men,Men
4,5,Animal,Lycan,Swimmer,Swimmer,Animal,Animal,-
5,6,Men,Men,Swimmer,Undead,Giant,Dragon,-
6,7,Animal,Animal,Animal,Undead,Dragon,-,-
7,8,Dragon,Dragon,Dragon,Dragon,Dragon,-,-


In [9]:
subTable = pd.read_csv('../../WildSubTable.csv')
subTable

,Die,Men Typical,Men Mountain,Men Desert,Men Water,Flyer,Undead,Giant,Lycan,Swimmer,Dragon,Animal
0,1,Bandit,Bandit,Nomad,Buccaneer,Pegasus,Skeleton,Kobold,Werewolf,Giant Crab,"Dragon, Black",Large Spider
1,2,Brigand,Berserker,Dervish,Pirate,Roc,Zombie,Goblin,Wereboar,Nixie,"Dragon, White",Giant Centipede
2,3,Necromancer,Brigand,Nomad,Buccaneer,Chimera,Ghoul,Orc,Weretiger,Giant Octopus,"Dragon, Green",Giant Lizard
3,4,Wizard,Lord,Lord,Pirate,Hippogriff,Wight,Hobgoblin,Werebear,Giant Squid,"Dragon, Blue",Giant Toad
4,5,Bandit,Wizard,Wizard,Buccaneer,Griffon,Wraith,Gnoll,-,Sea Monster,"Dragon, Red",Giant Ant
5,6,Berserker,Superhero,Nomad,Merman,Manticore,Mummy,Ogre,-,Giant Snake,"Dragon, Gold",Giant Weasel
6,7,Lord,Caveman,-,-,Roc,Spectre,Troll,-,Crocodile,Cockatrice,White Ape
7,8,Superhero,Necromancer,-,-,Wyvern,Vampire,Giant,-,Giant Leech,Basilisk,Giant Beetle
8,9,Brigand,Evil High Priest,-,-,Dragon,-,Gnome,-,Merman,Wyvern,Giant Scorpion
9,10,Bandit,Caveman,-,-,Pegasus,-,Dwarf,-,Nixie,Balrog,Lion


In [10]:
monsterTable = pd.read_csv('../../MonsterDatabase.csv')
monsterTable

,Monster,Number,AC,MV,HD,Lair%,Treas,Atk,Dam,Special,Type,Align,HD#,EHD,Env,Source
0,Rat,1d100,7,15,1/6,-,-,1,1,-,A,N,0.2,0,D,M2
1,Giant Centipede,2d12,9,15,1/3,-,-,1,0,Poison (-4),A,N,0.3,1,D,M1
2,Giant Rat,5d10,7,12,1/2,10,C,1,1d3,Disease,A,N,0.5,1,D,"S1, M1"
3,Jackal,1d6,8,12,1/2,-,-,1,1d3,-,A,N,0.5,1,W,"S1, M1"
4,"Giant Beetle, Fire",3d4,4,9,1-1,-,-,1,1d6,Light,A,N,0.8,1,D,S2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Will-O-Wisp,1,-8,18,9,1,A,1,2d6,"Light, Invisibility",X,N,9.0,27,D,S1
179,"Earth Elemental, Medium",1,2,6,12,-,-,1,3d6,MagicToHit (2),X,N,12.0,33,S,"O, S1"
180,"Fire Elemental, Large",1,2,12,16,-,-,1,2d6,MagicToHit (2),X,N,16.0,33,S,"O, S1"
181,"Water Elemental, Large",1,2,6,16,-,-,1,2d6,"Swimming (18), MagicToHit (2)",X,N,16.0,33,S,"O, S1"


In [12]:
terrainlist = mainTable.columns[1:].tolist()
terrainlist

['Clear', 'Woods', 'River', 'Swamp', 'Mountain', 'Desert', 'City']

In [16]:
# Get terrain and terrain index
random_index = random.randrange(len(terrainlist))
terrain = terrainlist[random_index]
terrainIndex = random_index + 1
terrain

'Swamp'

In [159]:
def monsterFixup(monName):
    #find giant subtype
    if monName == "Giant":
        monName == "Giant, Hill"
    d10 = random.randint(1,10)
    if d10 == 7:
        monName = "Giant, Stone"
        return monName
    if d10 == 8:
        monName = "Giant, Frost"
        return monName
    if d10 == 9:
        monName = "Giant, Fire";
        return monName
    if d10 == 10:
        monName = "Giant, Cloud"
        return monName

    # Find dragon subtype
    if monName == "Dragon":
        d6 = random.randint(1,6)
        if d6 == 1:
            monName = "Dragon, White"
            return monName
        if d6 == 2:
            monName = "Dragon, Black"
            return monName
        if d6 == 3:
            monName = "Dragon, Green";
            return monName
        if d6 == 4:
            monName = "Dragon, Blue"
            return monName
        if d6 == 5:
            monName = "Dragon, Red"
            return monName
        if d6 == 6:
            monName = "Dragon, Gold"
            return monName
    
    # Find specific giant animal
    if monName == "Giant Snake":
        monName = "Giant Snake, Constrictor"
        return monName
    if monName == "Giant Beetle":
        monName = "Giant Beetle, Bombardier"
        return monName
    if monName == "Giant Ant":
        monName = "Giant Ant, Worker"
        return monName
    if monName == "Sea Monster":
        monName = "Sea Monster, Small"
        return monName
    if monName == "Hydra":
        monName = "Hydra, 10 Heads"
        return monName
    if monName == "Roc":
        monName = "Roc, Small"	        
        return monName

    return monName

def getNPCEntourage():
    sumEnt = 0
    numEnt = random.randint(1,6) + random.randint(1,6)
    for i in range(numEnt):
        sumEnt += random.randint(1,4)
        #print(sumEnt)
    return sumEnt

def EHDFixup(monName, EHD):
    #Fill in null EHDs with estimated value.
    if EHD == 0:
        if monName == "Dragon, Gold":
            return 40
    return EHD
    
def handleNPCType(monsterName):
    NPClevel = 0
    if monsterName == "Wizard":
        NPClevel = 11
    if monsterName == "Necromancer":
        NPClevel = 10
    if monsterName == "Lord":
        NPClevel = 9
    if monsterName == "Superhero" or monsterName == "Patriarch" or monsterName == "Evil High Priest":
        NPClevel = 8
    if NPClevel == 0:
        return NPClevel
    else:
        return NPClevel + getNPCEntourage()
       
    
def rollEncounterByMonster(monsterName):
    # Special handling for NPC types
    NPCValue = handleNPCType(monsterName)
    print("NPCValue", NPCValue)
    if NPCValue > 0:
        return NPCValue
    
    #Find index in monster table
    monsterDice = monsterTable.loc[monsterTable["Monster"] == monsterName]["Number"].values[0]
    print(monsterDice)
    addition = 0
    multiplier = 1
    # RegEx code from @user1803551 on StackExchange:
    # http://stackoverflow.com/questions/35020687/
    # how-to-parse-dice-notation-with-a-java-regular-expression
    pattern = re.compile("([1-9]\\d*)?d([1-9]\\d*)([/x][1-9]\\d*)?([+-]\\d+)?")
    m = pattern.search(monsterDice)
    if m:
        print("m.groups", m.groups())
        dice = int(m.groups()[0])
        print("dice", dice)
        if m.groups()[1] is not None:
            sides = int(m.groups()[1])
            print("sides", sides)
        if m.groups()[2] is not None:
            multiplier = int(m.groups()[2].replace('x',''))
            print("multiplier", multiplier)
        if m.groups()[3] is not None:
            addition = int(m.groups()[3])
            print("addition", addition)
            
        monsterNumber = sum(random.randint(1,sides) for _ in range(dice))
        print(monsterNumber)
        monsterNumber = monsterNumber + addition
        print(monsterNumber)
        monsterNumber = monsterNumber * multiplier
        print(monsterNumber)
    else:
        monsterNumber = 1
        
    EHD = monsterTable.loc[monsterTable["Monster"] == monsterName]["EHD"].values[0].astype('int')
    print("EHDlookup", EHD)
    EHD = EHDFixup(monsterName, EHD)
    print("EHDfixup", EHD)
    
    totalEHD = monsterNumber * EHD
    print("totalEHD", totalEHD)
    hitDiceNum = monsterTable.loc[monsterTable["Monster"] == monsterName]["HD#"].values[0].astype('float32')
    print("hitDiceNum", hitDiceNum)
    if hitDiceNum <= 1.0:
        totalEHD = totalEHD / 4.0
        
    print("returntotalEHD", totalEHD)
    return totalEHD
        

def rollEncounterByTerrain():
    dice_type = (~mainTable[terrain].str.contains('-')).sum()
    subTableName = mainTable[terrain][0:dice_type].sample().values[0]
    print("subTable:", subTableName)
    subTableName == "Men"
    
    if subTableName == "Men":
        subTableName = "Men Typical"
    if terrain == "Mountain":
        subTableName = "Men Mountain"
    if terrain == "Desert":
        subTableName = "Men Desert"
    if terrain == "Water":
        subTableName = "Men Water"
    
    monster_dice_type = (~subTable[subTableName].str.contains('-')).sum()
    monsterName = subTable[subTableName][0:monster_dice_type].sample().values[0]
    monsterName = monsterFixup(monsterName)
    print("Monster:", monsterName)
    
    ehd = rollEncounterByMonster(monsterName)
    print("ENCOUNTER EHD", ehd)

In [160]:
rollEncounterByTerrain()

subTable: Men
Monster: Wizard
NPCValue 34
ENCOUNTER EHD 34
